In [ ]:
from pymongo import MongoClient
import pandas as pd
import numpy as np


In [ ]:
mongo_client = MongoClient("mongodb://192.168.1.214:27017/")
mongo_client.admin.command({"ping": 1})
mongo_client.server_info()
db = mongo_client["wowaudit_database"]
collection = db["wowaudit_hourly"]


In [ ]:
def filter_active_players(df):
    # Get the latest timestamp in the dataset
    last_timestamp = df['timestamp'].max()

    # Find players who have their last sample at this timestamp
    last_samples = df[df['timestamp'] == last_timestamp]
    active_players = last_samples['name'].unique()  # List of players who are still active

    # Append "Tojozomi" while ensuring uniqueness
    if "Tojozomi" not in active_players:
        active_players = np.append(active_players, "Tojozomi")
    print(active_players)

    # Filter the dataframe to only include active players
    filtered_df = df[df['name'].isin(active_players)]

    return filtered_df



In [ ]:
query_cols = ["timestamp", "metadata.name", "ilvl", "historical_mplus_done", "m+_score", "weathered_crests",
              "carved_crests", "runed_crests", "gilded_crests", "valorstones", "class", "season_mythic_dungeons",
              "season_delves", 'season_heroic_dungeons',
              "assembly_of_the_deeps_renown", "council_of_dornogal_renown", "hallowfall_arathi_renown",
              "severed_threads_renown",
              "week_heroic_dungeons", "week_delves", "week_mythic_dungeons", "wqs_this_week", "weekly_highest_m+",
              "arakara_city_of_echoes_total", "city_of_threads_total", "the_stonevault_total",
              "the_dawnbreaker_total", "grim_batol_total", "mists_of_tirna_scithe_total",
              "siege_of_boralus_total", "the_necrotic_wake_total"
              ]

query = ({c: 1 for c in query_cols})
query.update({"_id": 0})
data = collection.find({}, query)
df_raw = pd.DataFrame(list(data))

df = df_raw.copy()
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['name'] = df['metadata'].apply(lambda x: x.get('name', None))
df = df.drop(columns=['metadata'])
df = df.sort_values(by=['name', 'timestamp'])
df['ilvl_high_watermark'] = df.groupby('name')['ilvl'].cummax()
df = filter_active_players(df)
df[df["name"] == "Maggiesue"].sample(n=20, random_state=42)


In [ ]:
def process_historical_mplus_done_wide(df):
    # Define a helper function to parse the current week's dungeon data
    def parse_current_week_levels(field):
        if not isinstance(field, str) or not field:
            return {}

        # Split the string into weekly lists
        weekly_data = [
            list(map(int, week.split(','))) if week else []
            for week in field.split('|')
        ]

        # Get current week's data (last week in the list)
        current_week = weekly_data[-1] if weekly_data else []

        # Count occurrences of each dungeon level
        level_counts = {}
        for level in current_week:
            level_counts[level] = level_counts.get(level, 0) + 1

        return level_counts

    # Apply the function to parse the current week's dungeon levels
    dungeon_counts = df['historical_mplus_done'].apply(parse_current_week_levels)

    # Find all unique dungeon levels across the dataset
    all_levels = sorted(
        set(level for counts in dungeon_counts for level in counts.keys())
    )

    # Add a column for each dungeon level
    for level in all_levels:
        df[f'mplus_level_{level}'] = dungeon_counts.apply(lambda counts: counts.get(level, 0))

    return df


# Apply the function to your DataFrame
df = process_historical_mplus_done_wide(df)
df[df["name"] == "Maggiesue"].sample(n=20, random_state=42)


In [ ]:
CLASS_COLORS = {
    "Warrior": "#C79C6E",
    "Paladin": "#F58CBA",
    "Hunter": "#ABD473",
    "Rogue": "#FFF569",
    "Priest": "#FFFFFE",
    "Death Knight": "#C41F3B",
    "Shaman": "#0070DE",
    "Mage": "#69CCF0",
    "Warlock": "#9482C9",
    "Monk": "#00FF96",
    "Druid": "#FF7D0A",
    "Demon Hunter": "#A330C9",
    "Evoker": "#33937F",
}
df['class_color'] = df['class'].map(CLASS_COLORS)

# df['game_day'] = (df['timestamp'] - pd.Timedelta(hours=7)).dt.date

df[df["name"] == "Tojozomi"].sample(n=20, random_state=42)


In [ ]:
def calculate_reset_timestamps(df):
    # Sort by player name and timestamp
    df = df.sort_values(by=['name', 'timestamp'])

    # Weekly cumulative sum columns
    weekly_cols = [
        'week_heroic_dungeons',
        'week_delves',
        'week_mythic_dungeons',
        'wqs_this_week',
        'weekly_highest_m+'
    ]

    # Step 1: Identify reset timestamps per player
    player_resets = []

    for player, player_df in df.groupby('name'):
        prev_row = None
        resets = []

        for idx, row in player_df.iterrows():
            ts = row['timestamp']
            is_tuesday = ts.weekday() == 1  # Tuesday is 1

            if prev_row is not None and is_tuesday:
                # Detect resets based on weekly sums decreasing
                if any(row[col] < prev_row[col] for col in weekly_cols if not pd.isna(row[col])):
                    resets.append(ts)

            prev_row = row

        # Keep only the latest reset timestamp for each Tuesday
        if resets:
            latest_resets = {}
            for reset in resets:
                reset_date = reset.date()
                if reset_date not in latest_resets or reset > latest_resets[reset_date]:
                    latest_resets[reset_date] = reset
            player_resets.extend([(player, reset_date, reset) for reset_date, reset in latest_resets.items()])

    # Convert player resets to a DataFrame
    reset_df = pd.DataFrame(player_resets, columns=['name', 'reset_date', 'reset_timestamp'])

    # Step 2: Confirm consistency across players
    consistency_check = reset_df.groupby('reset_date')['reset_timestamp'].nunique()

    inconsistent_dates = consistency_check[consistency_check > 1]
    if not inconsistent_dates.empty:
        print("Inconsistent reset times detected for the following dates:")
        print(inconsistent_dates)

    return reset_df


# Apply the function to detect reset timestamps
reset_df = calculate_reset_timestamps(df)
reset_df



In [ ]:
def calculate_game_time_from_resets(df, reset_df):
    # Consolidate reset timestamps into a dictionary (reset_date -> reset_timestamp)
    reset_timestamps = (
        reset_df.groupby('reset_date')['reset_timestamp']
        .first()  # Take the first (or only) timestamp if consistent
        .sort_values()
        .to_dict()
    )
    print(reset_timestamps)

    # Sort the DataFrame by timestamp
    df = df.sort_values(by='timestamp')

    # Calculate the default reset date: the Tuesday at 7 AM before the first timestamp
    first_timestamp = df['timestamp'].min()
    days_since_tuesday = (first_timestamp.weekday() - 1) % 7  # How far back Tuesday is
    default_reset_date = (first_timestamp - pd.Timedelta(days=days_since_tuesday)).replace(
        hour=7, minute=0, second=0, microsecond=0
    )

    # Initialize columns
    df['game_week'] = None
    df['game_day'] = None

    # Track the current reset date
    last_reset_date = default_reset_date.date()

    for idx, row in df.iterrows():
        ts = row['timestamp']
        day_of_week = ts.weekday()  # 0 = Monday, 1 = Tuesday, ..., 6 = Sunday

        # Update the reset date if it's a Tuesday and a reset is defined
        if day_of_week == 1:  # Tuesday
            reset_date = ts.date()
            if reset_date in reset_timestamps:
                if ts >= reset_timestamps[reset_date]:
                    last_reset_date = reset_date

        # Assign the game week (using the last reset date)
        game_week = last_reset_date

        # Determine the start of the game day
        if day_of_week == 1:
            # Use the reset time for Tuesday
            game_day_start = reset_timestamps[ts.date()]
        else:
            # Use 7 AM for all other days
            game_day_start = ts.replace(hour=7, minute=0, second=0, microsecond=0)

        # Determine game day
        if ts < game_day_start:
            # Belongs to the previous game day
            game_day = game_day_start - pd.Timedelta(days=1)
        else:
            game_day = game_day_start

        # Assign values
        df.at[idx, 'game_week'] = game_week
        df.at[idx, 'game_day'] = game_day.date()

    return df


df = calculate_game_time_from_resets(df, reset_df)
df


In [ ]:
def calculate_season_total_from_historical(df):
    # Helper function to parse and count dungeons from historical_mplus_done
    def count_dungeons_from_historical(field):
        if not isinstance(field, str) or not field:
            return 0

        # Split the field by '|' into weekly data
        weekly_data = [week.split(',') for week in field.split('|') if week]

        # Count all dungeon entries across weeks
        total_dungeons = sum(len(week) for week in weekly_data)
        return total_dungeons

    # Apply the function to calculate the running season total
    df['corrected_season_mythic_dungeons'] = df['historical_mplus_done'].apply(count_dungeons_from_historical)

    return df


# Apply the function to compute the new season total
df = calculate_season_total_from_historical(df)

# Display a sample of the updated DataFrame
df[df["name"] == "Maggiesue"][['timestamp', 'historical_mplus_done', 'season_mythic_dungeons', 'week_mythic_dungeons',
                               'corrected_season_mythic_dungeons']]